In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

list_dir = '/content/gdrive/My Drive/Group_Project/Face-Mask-Detection-main_UnzippedFiles/Face-Mask-Detection-main'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
# we applied less learning rate because less learning rate means loss will be calculated properly.
INIT_LR = 0.0001
EPOCHS = 15
BS = 32

In [ ]:

DIRECTORY = (f"{list_dir}/dataset1/experiements/data")
CATEGORIES = ["with_mask", "without_mask"]

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

[INFO] loading images...


In [ ]:
data = []
labels = []

In [ ]:
# withmask and without mask images will append to data array and categories will append to labels array
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224)) #all images will be 224 x 224
    	image = img_to_array(image) #images to array using imagetoarray function
    	image = preprocess_input(image) # mobileNet used for model so, we used preprocess_input fuction

    	data.append(image)
    	labels.append(category) #with_mask and without_mask

In [ ]:
# data is numerical values, but labels are alphbetic variables, withmask and withoutmask.
# We should convert them to numerical values by using LabelBinarizer() method. that comes from the sklearn.Preprocesser model

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# we converted them into numerical ones (0 , 1 ...)

In [ ]:
# we need to convert them
data = np.array(data, dtype="float32")
labels = np.array(labels)

#train & test data 0.2 for testing and 0.8 for training
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [ ]:
# construct the training image generator for data augmentation
# ImageDataGenerator() creates many images from single image
# by changing some of the properties like rotating, shifting etc.
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3))) #it means height 224, weight 224, \and 3 channels R,G & B

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model

headModel = baseModel.output
headModel = MaxPooling2D(pool_size=(6, 6))(headModel) #creating the pooling 7 by 7
headModel = Flatten(name="flatten")(headModel) #doing flatten this layers
headModel = Dense(128, activation="relu")(headModel) # dense layer and relu is non-linear use cases.
headModel = Dropout(0.5)(headModel) #dropout for avoiding
headModel = Dense(2, activation="softmax")(headModel)

In [ ]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = True

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] training head...
Epoch 1/15
34/34 [==============================] - 23s 447ms/step - loss: 1.4217 - accuracy: 0.8839 - val_loss: 6.5650 - val_accuracy: 0.5000
Epoch 2/15
34/34 [==============================] - 13s 390ms/step - loss: 0.0779 - accuracy: 0.9738 - val_loss: 11.3916 - val_accuracy: 0.5000
Epoch 3/15
34/34 [==============================] - 14s 397ms/step - loss: 0.0979 - accuracy: 0.9779 - val_loss: 7.4682 - val_accuracy: 0.5000
Epoch 4/15
34/34 [==============================] - 14s 409ms/step - loss: 0.0552 - accuracy: 0.9850 - val_loss: 8.3923 - val_accuracy: 0.5000
Epoch 5/15
34/34 [==============================] - 13s 385ms/step - loss: 0.2064 - accuracy: 0.9522 - val_loss: 2.1302 - val_accuracy: 0.5000
Epoch 6/15
34/34 [==============================] - 13s 386ms/step - loss: 0.1409 - accuracy: 0.9494 - val_loss: 6.7965 - val_accuracy: 0.5000
Epoch 7/15
34/34 [==============================] - 13s 389ms/step - loss: 0.2234 - accuracy: 0.9242 - val_loss: 0.74

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

In [ ]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector.model", save_format="h5")


In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")